In [1]:
import pandas as pd
import datetime
import json

In [2]:
channel = "Scikitlearn"
channel2 = "Angular"

## Read users

In [2]:
df = pd.read_json('../Gitter_Channels/Scikitlearn/Scikitearn_user.json')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4318 entries, 0 to 4317
Data columns (total 3 columns):
displayName    4318 non-null object
id             4318 non-null object
username       4318 non-null object
dtypes: object(3)
memory usage: 134.9+ KB


In [4]:
df.head()

,displayName,id,username
0,Jitesh Khandelwal,529c6a18ed5ab0b3bf04d26a,jiteshk23
1,Rahul Kulhari,529c6bdded5ab0b3bf04d758,RahulKulhari
2,Jaepil Jeong,529c6c0bed5ab0b3bf04d7db,jaepil
3,Edwin Khoo,529c6c8eed5ab0b3bf04d94a,edwinksl
4,Thanh Nguyen,529c6c9bed5ab0b3bf04d96b,tienthanh8490


## Read chat logs

In [3]:
import json

In [4]:
save_path = "../Gitter_Channels/"

In [10]:
def read_logs(utterance_list):
    prev_user = ""
    dialog_id = 0
    results = []
    profile = {}
    isHead =False
    create_time = ""
    line = 0
    
    for utterance in utterance_list:     
        if len(utterance) == 0:
            continue
        if '-------------------------------------' in utterance:
            if dialog_id != 0:
                issue["Id"] = dialog_id
                #print(issue["Id"])
                issue["asker"] = prev_user
                #print(issue["asker"])
                issue["time"] = create_time
                #print(issue["time"])
                issue["content"] = content
                #print(issue["content"])
                issue["lines"] = line
                #print(issue["lines"])
                results.append(issue)
            dialog_id += 1
            line = 0
            issue = {}
            content = ""
            isHead =True
        else:
            line += 1
            start_index = 0
            end_index = 0
            #retreive time
            while utterance[start_index] != "[":
                start_index += 1
        
            start_index += 1
            end_index = start_index
            while utterance[end_index] != "]":
                end_index += 1
        
            time = utterance[start_index:end_index]
        
            #retreive user
            start_index = end_index
            while utterance[start_index] != "<":
                start_index += 1
        
            start_index += 1
            end_index = start_index
            while utterance[end_index] != ">":
                end_index += 1
            
            curr_user= utterance[start_index:end_index]
        
            #retreive message
            message = utterance[(end_index+2):]
            #print("message:",message)

            
            if line == 1:
                prev_user = curr_user
                create_time = time
            
            if isHead and prev_user == curr_user:
                content += message
            else:
                isHead =False
                if prev_user == curr_user:
                    continue
            
                if len(profile) > 0 and curr_user in profile.keys():
                    if dialog_id in profile[curr_user].keys():
                        profile[curr_user][dialog_id]["Answer"] += message
                    else:
                        profile[curr_user][dialog_id] = {}
                        profile[curr_user][dialog_id]["Time"] = time
                        profile[curr_user][dialog_id]["Answer"] = message
    
                else:
                    profile[curr_user] = {}
                    profile[curr_user][dialog_id] = {}
                    profile[curr_user][dialog_id]["Time"] = time
                    profile[curr_user][dialog_id]["Answer"] = message
    
    return results, profile

In [22]:
def read_msg(utterance_list):
    dialog_id = 0
    results = []
    isHead =True
    
    for utterance in utterance_list:    
        if len(utterance) == 0:
            continue
            
        if '-------------------------------------' in utterance:
            dialog_id += 1
            isHead =True
            continue
        
        dialog = {}  
        dialog["Id"] = dialog_id
        start_index = 0
        end_index = 0
        
        #retreive time
        while utterance[start_index] != "[":
            start_index += 1
            
        start_index += 1
        end_index = start_index
        while utterance[end_index] != "]":
            end_index += 1
            
        dialog["date"] = utterance[start_index:end_index]
        
        #retreive user
        start_index = end_index
        while utterance[start_index] != "<":
            start_index += 1
        
        start_index += 1
        end_index = start_index
        while utterance[end_index] != ">":
            end_index += 1
        
        curr_user= utterance[start_index:end_index]
        
        dialog["asker"] = curr_user
        
        
        #determine type 
        if isHead:
            dialog["type"] = "issue"
        elif results[len(results)-1]["type"] == "issue" and results[len(results)-1]["asker"] == curr_user:
            dialog["type"] = "issue"
        else:
            dialog["type"] = "answer"
        
        #retreive message
        dialog["content"] = utterance[(end_index+2):]

        results.append(dialog)
        isHead =False
    
    return results, dialog_id

In [19]:
with open(save_path+channel2+"/disentangle/test.txt", "r") as f:
        utterance_list = f.readlines()

In [23]:
augular_results, num_dialog = read_msg(utterance_list)

In [24]:
num_dialog

15

In [26]:

df = pd.read_json(json.dumps(augular_results))

In [27]:
df

,Id,date,asker,type,content
0,0,2015-03-07 19:32:00,530cd1eb5e986b0712efb4d2,issue,Welcome all to Angular 2 public discussion!\n
1,1,2015-03-07 21:41:00,54c89b1cdb8155e6700f2ac3,issue,Awesome!\n
2,2,2015-03-07 21:53:00,54e6ee3b15522ed4b3dc3c24,issue,"sweet, now we can ask angular2 questions in re..."
3,3,2015-03-07 22:04:00,530cd1eb5e986b0712efb4d2,issue,Yes!\n
4,4,2015-03-07 22:06:00,54e6ee3b15522ed4b3dc3c24,issue,we're missing the [View Source] [Improve this ...
5,5,2015-03-07 23:18:00,530cd1eb5e986b0712efb4d2,issue,Good idea can you file an issue with repo?\n
6,6,2015-03-08 00:20:00,54e6ee3b15522ed4b3dc3c24,issue,I submitted it an issue about view source etc ...
7,7,2015-03-08 05:45:00,54f9f23015522ed4b3dce8b5,issue,"Great job on Angular 2 so far! IMHO, the Angul..."
8,8,2015-03-08 05:56:00,54e6ee3b15522ed4b3dc3c24,issue,"So your answer is yes, eventually. TypeScript ..."
9,8,2015-03-08 05:58:00,54e6ee3b15522ed4b3dc3c24,issue,here is the link to the typescript todo app de...


In [ ]:
for did in range(num_dialog):
    df_select = df[df["Id"] == did]

### Scikitlearn

In [28]:
with open("../Gitter_Channels/"+channel+"/disentangle/"+channel+"_result.txt", "r") as f:
        utterance_list = f.readlines()

In [ ]:
scikitlearn_results,scikitlearn_profile = read_logs(utterance_list)

In [91]:
len(scikitlearn_results)

5414

In [ ]:
with open(save_path+channel+'/Scikitlearn_issue.json', 'w') as outfile:
    json.dump(scikitlearn_results, outfile)
with open(save_path+channel+'/Scikitlearn_profile.json', 'w') as outfile:
    json.dump(scikitlearn_profile, outfile)

### Angular

In [30]:
with open(save_path+channel2+"/disentangle/"+channel2+"_result.txt", "r") as f:
        utterance_list = f.readlines()

In [11]:
augular_results,angluar_profile = read_logs(utterance_list)

In [12]:
len(augular_results)

422974

In [9]:
with open(save_path+channel2+'/Angular_issue.json', 'w') as outfile:
    json.dump(augular_results, outfile)
with open(save_path+channel2+'/Angular_profile.json', 'w') as outfile:
    json.dump(angluar_profile, outfile)

In [31]:
augular_results, num_dialog = read_msg(utterance_list)

In [32]:
num_dialog

422975

In [33]:
df = pd.read_json(json.dumps(augular_results))
df

,Id,date,asker,type,content
0,0,2015-03-07 19:32:00,530cd1eb5e986b0712efb4d2,issue,Welcome all to Angular 2 public discussion!\n
1,1,2015-03-07 21:41:00,54c89b1cdb8155e6700f2ac3,issue,Awesome!\n
2,2,2015-03-07 21:53:00,54e6ee3b15522ed4b3dc3c24,issue,"sweet, now we can ask angular2 questions in re..."
3,3,2015-03-07 22:04:00,530cd1eb5e986b0712efb4d2,issue,Yes!\n
4,4,2015-03-07 22:06:00,54e6ee3b15522ed4b3dc3c24,issue,we're missing the [View Source] [Improve this ...
...,...,...,...,...,...
1135246,422971,2022-08-18 14:26:00,5ea477add73408ce4fe1af23,issue,Hi there. A short question. Is there any benef...
1135247,422972,2022-08-18 15:13:00,62fe56da6da03739849b8211,issue,[![MicrosoftTeams-image (1).png](https://files...
1135248,422972,2022-08-18 15:13:00,62fe56da6da03739849b8211,issue,can someone help me achieve this using materia...
1135249,422973,2022-08-19 03:23:00,matrix-sharonmary:matrix.org,issue,Good news Put an end to your financial proble...


In [38]:
df[(df["date"].year == 2022) & (df["date"].month == 6) & (df["date"].day == 21) ]

AttributeError: 'Series' object has no attribute 'year'

In [37]:
df.loc[0]["date"].day

7

## Read issues from ISPY

In [4]:
with open("../ISPY_data/angular_ispair.txt", "r",encoding = "utf-8") as f:
        utterance_list = f.readlines()

In [5]:
len(utterance_list)

19986

In [30]:
#tags = ["[<-CODE->]","[<-ISSUE->]","[<-LINK->]"]
issues = []
solutions = []
skip = False
for index, sentence in enumerate(utterance_list):
    sentence =sentence.replace("[<-CODE->]","")
    sentence =sentence.replace("[<-ISSUE->]","")
    sentence =sentence.replace("[<-LINK->]","")
        
    
    #issue
    if (index+1) % 3 == 1:
        if len(sentence) < 25:
            skip = True
            continue
            
        issues.append(sentence[7:].strip())
        skip = False
    elif (index+1) % 3 == 2 and not skip:
        solutions.append(sentence[10:].strip())

In [31]:
len(issues)

5947

In [32]:
len(solutions)

5947

In [33]:
df= pd.DataFrame()
df["issue"] = issues
df["solution"] = solutions
df

,issue,solution
0,there isn't a.just()in Rx5 note that that's ...,Check out this . This is the documentation for...
1,Shouldn'tDynamicComponentLoader#loadIntoLocati...,gionkunz: so how could i add it inside the div...
2,Will there be a material stuff for angular 2?,Currently I am using . I wonder if I am going...
3,Has anyone run into a situation where ng2 inst...,justindujardin: no.. are you sure it doesn't m...
4,Who hereplans on writing all their angular 2 a...,seanpar203: the biggest issue in using pure JS...
...,...,...
5942,that's a new one :),
5943,I'm running angular 8.2.14 wanting to migrate ...,"I saw no example of that, even I saw lot of un..."
5944,East coast here too. No celebrating just yet XD,
5945,so your api is on port 8080? your http call sh...,I set up a docker environment and nginx proxy ...


In [35]:
df.to_csv("C:/Users/wangs/Desktop/lucene/angular_ispy.csv")

## Convert to csv

In [7]:
def strToDatetime(str):
    strptime = datetime.datetime.strptime(str, "%Y-%m-%d %H:%M")
    return strptime

In [8]:
def conv_to_dateframe(profile):
    user_Id = []
    cluster_Id = []
    answer_date = []
    answers = []

    for user in profile:
        for cluster in profile[user]:
            #print(profile[user][cluster]["Time"])
            time = strToDatetime(profile[user][cluster]["Time"])
            user_Id.append(user)
            cluster_Id.append(cluster)
            answer_date.append(time)
            answers.append(profile[user][cluster]["Answer"])
    
    df_user = pd.DataFrame()
    df_user["user_Id"] =user_Id
    df_user["cluster_Id"] =cluster_Id
    df_user["answer_date"] =answer_date
    df_user["answers"] =answers
    
    return df_user

### Scikitlearn

In [99]:
with open(save_path+channel+'/'+channel+'_profile.json', 'r') as f:
    profile = json.load(f)

In [104]:
len(profile)

106

In [105]:
df_scikitlearn_profile = conv_to_dateframe(profile)

In [107]:
df_scikitlearn_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54d4a1d6db8155e6700f853b,12,2015-02-06 11:14:00,I'm with family this week and so not super act...
1,54d4a1d6db8155e6700f853b,61,2015-02-24 22:46:00,cool\nok\n
2,54d4a1d6db8155e6700f853b,69,2015-02-25 18:59:00,yeah that sounds like a good idea\n
3,54d4a1d6db8155e6700f853b,72,2015-02-25 19:14:00,you mean == 1\n
4,54d4a1d6db8155e6700f853b,75,2015-02-25 19:19:00,that is also weird ;)\n


In [108]:
df_scikitlearn_profile.to_csv(save_path+channel+'/'+channel+'_profile.csv', index=False) 

### Angular

In [9]:
with open(save_path+channel2+'/'+channel2+'_profile.json', 'r') as f:
    profile = json.load(f)

In [10]:
len(profile)

6707

In [11]:
df_angular_profile = conv_to_dateframe(profile)

In [12]:
df_angular_profile.head()

,user_Id,cluster_Id,answer_date,answers
0,54e6ee3b15522ed4b3dc3c24,15,2015-03-08 18:01:00,currently only working in chrome until shadow ...
1,54e6ee3b15522ed4b3dc3c24,20,2015-03-09 02:39:00,::shadow\nhttp://www.html5rocks.com/en/tutoria...
2,54e6ee3b15522ed4b3dc3c24,100,2015-03-13 19:48:00,@mhevery that was my worry too (Dart version ...
3,54e6ee3b15522ed4b3dc3c24,113,2015-03-14 02:46:00,you donat need to inject it since itas going t...
4,54e6ee3b15522ed4b3dc3c24,118,2015-03-14 07:23:00,what version of node?\n


In [ ]:
df_angular_profile.to_csv(save_path+channel2+'/'+channel2+'_profile.csv', index=False) 